In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import njit
import jitr

In [2]:
nchannels = 3
Ecom = 14.1
nodes_within_radius = 5

sys = jitr.ProjectileTargetSystem(
    np.ones(nchannels) * 939.0,
    np.ones(nchannels) * nodes_within_radius * (2 * np.pi),
    l=np.ones(nchannels, dtype=np.int64) * 0,
    level_energies=np.array([0.0, 10.0, 11.0]),
    Ztarget=40,
    Zproj=0,
    nchannels=nchannels,
)

/home/kyle/mambaforge/envs/om/lib/python3.10/site-packages/numba/core/dispatcher.py:289: UserWarning: Numba extension module 'numba_scipy' failed to load due to 'ValueError(No function '__pyx_fuse_0pdtr' found in __pyx_capi__ of 'scipy.special.cython_special')'.
  entrypoints.init_all()


In [3]:
# Lagrange-Mesh solver
solver = jitr.LagrangeRMatrixSolver(50, 3, sys, ecom=14.1)

In [4]:
# set up diagonal part of interaction
# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 0  # imag potential strength- we will set to 0 to verify that the S-matrix is unitary
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness

params = (V0, W0, R0, a0)

interaction_matrix = jitr.InteractionMatrix(sys.nchannels)
for i in range(sys.nchannels):
    interaction_matrix.set_local_interaction(
        jitr.woods_saxon_potential, i, i, args=params
    )

In [5]:
# set up coupling terms
params_off_diag = (V0 / 2, W0 / 2, R0, a0)
for i in range(sys.nchannels):
    for j in range(sys.nchannels):
        interaction_matrix.set_local_interaction(
            jitr.woods_saxon_potential, i, j, params_off_diag
        )

In [6]:
channels = sys.build_channels(Ecom)

In [14]:
%%timeit
R, S, uext_prime_boundary, x = solver.solve(
    interaction_matrix, channels, wavefunction=True
)

4.38 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
S

array([[-1.09179845+0.41686829j, -2.09179845+0.41686829j,
        -2.09179845+0.41686829j],
       [-0.64894996+0.12932731j,  0.35105004+0.12932731j,
        -0.64894996+0.12932731j],
       [ 0.81714493-0.16284638j,  0.81714493-0.16284638j,
         1.81714493-0.16284638j]])

In [12]:
assert np.isclose(jitr.complex_det(S), 1.0)